In [33]:
import matplotlib
import numpy as np
import torch 
import torch.nn as nn
import torch.nn.functional as F
import math
import tqdm as tq
from tinygrad.nn.datasets import mnist
from matplotlib import pyplot as plt

In [38]:
X_train, Y_train, X_test, Y_test = mnist()
X_train = X_train.numpy().astype(np.float32)
Y_train = Y_train.numpy().astype(np.int8)
X_test = X_test.numpy().astype(np.float32)
Y_test = Y_test.numpy().astype(np.int8)

In [35]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,6,4)
        self.conv2 = nn.Conv2d(6,16,4)
        self.linear = nn.Linear(256, 10) #bs is 256
    def forward(self,x):
        c1 = F.relu(self.conv1(x)) #relu activation function after first convolution layer
        l2 = F.max_pool2d(c1, (2,2))#subsampling, breaking down to process better
        c2 = F.relu(self.conv2(l2)) #second convolution
        l4 = F.max_pool2d(c2,(2,2))
        l4 = torch.flatten(l4,1) #flatten down to one dimension to run through fully connected layer/mlp
        out = self.linear(l4)
        return out
model = ConvNet()

In [ ]:
#run 

optim = torch.optim.Adam(model.parameters(), lr=0.005)
criterion = nn.CrossEntropyLoss()
batch_size = 256
for i in range(100):
    samp = np.random.randint(0, X_train.shape[0], size=(batch_size))
    X = torch.tensor(X_train[samp])
    Y = torch.tensor(Y_train[samp]).long()

    optim.zero_grad() #reset gradients
    out = model(X) #forward pass
    cat = torch.argmax(out, dim=1) 
    #print("cat",cat.shape)
    accuracy = (cat == Y).float().mean() #check how many are right
    #run backprop
    loss = loss_func(out, Y)

    loss.backward()
    #do grad descent 
    
    optim.step()
    loss, accuracy = loss.item(), accuracy.item()
    accuracies.append(accuracy)
    losses.append(loss)
    if k % 100 == 0:
        print("loss %.2f accuracy %.2f" % (loss, accuracy))
plt.ylim(-1,2)
plt.plot(losses)
plt.plot(accuracies)